# GtR enriched exploration

In this notebook we explore the results of the enrichment of the GtR data using three sets of labels:

* Academic disciplines, based on a model trained on a labelled subset of the GtR data
* Industries, based on a model trained on a corpus of business website data
* SDGs based on a labelled corpus of SDG related documents.

We will load the data, perform an analysis of salient terms, explore correlations between enriched variables and with other metadata available etc.






## 0. Preamble

In [ ]:
%run notebook_preamble.ipy

In [ ]:
import string as st
import wordcloud
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

import matplotlib.pyplot as plt

In [ ]:
# Put functions and things here

def get_latest_file(date_list,date_format='%d-%m-%Y'):
    '''
    This function takes a list of date strings and returns the most recent one
    
    Args:
        date_list: a list of strings with the format date_filename
        date_format: the format for the date, defaults to %d-%m-%Y
    
    Returns:
        The element with the latest date
    
    '''
    
    #This gets the maximum date in the gtr directory
    dates = [datetime.datetime.strptime('-'.join(x.split('_')[:3]),date_format) for x in date_list]
    
    #Return the most recent file
    most_recent = sorted([(x,y) for x,y in zip(date_list,dates)],key=lambda x:x[1])[-1][0]
    
    return(most_recent)
                                   

def flatten_list(my_list):
    '''
    Flattens a list
    '''
    
    return([x for el in my_list for x in el])
    


In [ ]:
class SalientWords():
    '''
    Class that extracts salient words from clusters of data.
    
    Arguments:
        A dataframe and two strings (the variable to groupby and the variable to use as text)
        
    Methods:
        .count_vect(): word frequencies for all words (takes **kwargs for additional parameters in the count vectorisation)
        .salient(): tfidf. It will also take **kwargs and a threshold for including words in the results
        .visualise(): it visualises the data as wordclouds
    
    '''
    
    def __init__(self,df,categories,text):
        '''
        Initialises with key variables
        
        '''    
        
        
        #This creates the joined corpus
        self.grouped_corpus = df.groupby(categories)[text].apply(lambda x: ' '.join(x))
        
        
        #Remove digits and special 
        dig = r'|'.join(st.digits)
        out = '\W+'
        
        self.processed_text = [re.sub(out,' ',re.sub(dig,' ',x.lower())) for x in self.grouped_corpus]
        
        #This is a dict we will use to store the results later
        self.groups = {i:[] for i in self.grouped_corpus.index}
        
        #return(self)
        
    def word_freqs(self,**kwargs):
        '''
        Terms frequencies over categories
        
        '''
        #load corpus
        X = self.processed_text
        
        count_vect = CountVectorizer(**kwargs)
        
        #Store outputs
        self.count_vect = count_vect
        self.token_freqs = count_vect.fit_transform(X)
        
        return(self)
    
    def salient(self,min_threshold=1000,extra_stops=['research','project','new','projects'],**kwargs):
        '''
        Salient terms in the data.
        
        '''
        
        #Get selected words
        
        word_freqs = pd.DataFrame(self.token_freqs.todense(),columns=self.count_vect.get_feature_names())
        
        word_freqs_total = word_freqs.sum(axis=0)
        
        #Create a dict so we can put word frequencies together with salient words later
        #word_freqs_dict = word_freqs.to_dict()
        
        #I am interested in any words above the threshold
        my_words = [x for x in word_freqs_total.index[word_freqs_total>min_threshold] if x not in extra_stops]
        
        
        #Initialise the tfidf
        tf = TfidfTransformer(**kwargs)
        
        
        #out
        X = tf.fit_transform(self.token_freqs)
        
        X_selected = pd.DataFrame(X.todense(),columns=self.count_vect.get_feature_names())[my_words]
            
            
        #Store the results
        for n,x in enumerate(self.groups.keys()):
            
            #Creates the dataframe combining tfs and wfs
            result = pd.concat([X_selected.iloc[n],word_freqs.iloc[n][my_words]],axis=1)
            
            result.columns = ['tfidf','count'] 
                   
            self.groups[x] = result
            
        return(self)
        
    def get_summary(self,tf_threshold=90,wf_threshold=75):
        '''
        
        Extracts a summary of the data based on tf and wf thresholds
        
        '''
        
        self.summary={i:[] for i in self.groups.keys()}
        
        for x in self.groups.keys():
            
            #Creates the dataframe
            result = self.groups[x]
            
            tf_thres = np.percentile(result['tfidf'],tf_threshold)
            
            summary = result.loc[result['tfidf']>tf_thres]
            
            wf_thres = np.percentile(result['count'],wf_threshold)
            
            summary_2 = summary.loc[summary['count']>wf_thres]
                   
            self.summary[x] = summary_2.sort_values('tfidf',ascending=False)
        
        return(self)
    
def make_wordcloud(term_freqs_df,var,name,ax):
    '''
    This function takes a df generated by the SalientWords class and returns a wordcloud
    
    '''
    
    input_dict = {w:f for w,f in zip(term_freqs_df.index,term_freqs_df[var])}

    wc = wordcloud.WordCloud(background_color="black").generate_from_frequencies(input_dict)

    ax.imshow(wc)
    
    ax.axis('off')
    
    ax.set_title(name)

## 1. Load data

In [ ]:
#Note that this also includes the discipline predictions

sector_df = pd.read_csv('../data/processed/21_5_2019_gtr_with_industry_labels.csv',compression='zip')

In [ ]:
sdg_df = pd.read_csv('../data/processed/2_5_2019_gtr_sdg_labelled.csv',compression='zip')

In [ ]:
#Combine both df avoding repeated columns
combined = pd.concat([sector_df,sdg_df[[x for x in sdg_df.columns if x not in sector_df.columns]]],axis=1)

Lists of category elements for analysis

In [ ]:
disc_list = [x for x in combined.columns if 'disc_' in x]
industry_list = [x for x in combined.columns if any(sect in x for sect in ['primary','construction','manufacture','services'])]
sdg_list = [x for x in combined.columns if 'sdg_' in x]

In [ ]:
combined.columns = ['ind_'+x if x in industry_list else x for x in combined.columns]

industry_list = ['ind_'+x for x in industry_list]

## 2. Check data

### Disciplines

In [ ]:
from scipy.stats import kurtosis

def prediction_diagnostics(df,predicted_list,ax,prob_threshold=0.05):
    '''
    This function runs a bunch of tests with the vectors of predictions we have loaded.
    
    This includes:
    
    
    -Create a df with the variables.
    -Remove noisy predictions (below threshold)
    -calculate the variance of predictions per observation and plot it.
    -label predictions which are tight (in the top quartile of variance for each category)
    -calculate kurtosis and label observations with high curtosis
    -label predictions which are in the top quartile for the total and a category
    
    '''
    
    #Remove probabilities below threshold
    my_df = df[predicted_list].copy().applymap(lambda x: 0 if x<prob_threshold else x)
    
    #Diagnostics df
    diag_df = pd.DataFrame()
    
    #Maximum prediction
    
    diag_df['max_pred'] = my_df.max(axis=1)
    
    #top category in obs
    diag_df['top_category'] = my_df.idxmax(axis=1)
    
    #Calculate variance
    diag_df['prediction_variance'] = my_df[predicted_list].apply(lambda x: np.var(x),axis=1)

    diag_df.groupby('top_category')['prediction_variance'].mean().sort_values(ascending=False).plot.bar(title='Mean variance in predictins by top category',color='blue',
                                                                                                                           ax=ax[0])

    
    #Is a variable in the prediction quartile
    pred_variance_quartile = diag_df.groupby('top_category')['prediction_variance'].apply(lambda x: np.percentile(x,75))
    
    pred_variance_quartile_all = np.percentile(diag_df['prediction_variance'],75)
    
    diag_df['tight_prediction']= [x>pred_variance_quartile[cat] for x,cat in zip(diag_df['prediction_variance'],diag_df['top_category'])]
    
    diag_df['tight_prediction_all']= [x>pred_variance_quartile_all for x in diag_df['prediction_variance']]
    
    pd.crosstab(diag_df['top_category'],diag_df['tight_prediction_all'],normalize=1).plot.bar(title='Tight predictions by category',ax=ax[1])
    
    
    #Kurtosis
    
    diag_df['kurtosis'] = my_df.apply(lambda x: kurtosis(x),axis=1)
    
    #Kurtosis plot
    
    diag_df.groupby('top_category')['kurtosis'].mean().sort_values(ascending=False).plot.bar(color='blue',title='Mean kurtosis by top category',ax=ax[2])
    
    
    #Percentiles in predicted values
    pc_75_preds_all = np.percentile(flatten_list([my_df.loc[my_df[cat]>0,cat] for cat in predicted_list]),95)

    #Are any of the predictions for a project above the 75 pc for all predictions?
    diag_df['has_top_pred']= my_df[predicted_list].apply(lambda x: any(v>pc_75_preds_all for v in x),axis=1)
    
    #Prediction percentile per sector
    pc_75_by_cat = {cat: np.percentile(my_df.loc[my_df[cat]>0,cat],95) for cat in predicted_list}

    pd.DataFrame(pc_75_by_cat,index=[0]).T.sort_values(0,ascending=False).plot.bar(color='blue',legend=False,title='75 pc probability',ax=ax[3])
    
    df_quarts = pd.DataFrame()
    
    for cat in predicted_list:
        df_quarts[cat+'_top_q'] = my_df[cat]>pc_75_by_cat[cat]
    
    return([diag_df,df_quarts])

In [ ]:
fig,ax = plt.subplots(figsize=(7,15),nrows=4)

disc_diag = prediction_diagnostics(combined,disc_list,prob_threshold=0.1,ax=ax)

plt.tight_layout()

In [ ]:
combined['disc_top'] = disc_diag[0]['top_category']

In [ ]:
sal_disc = SalientWords(combined,categories='disc_top',text='abstract')
sal_disc.word_freqs(**{'stop_words':'english','max_features':2000,'ngram_range':(1,2)}).salient(min_threshold=500).get_summary(wf_threshold=50)

In [ ]:
fig,ax = plt.subplots(ncols=2,nrows=4,figsize=(10,10))

for n,name in enumerate(sal_disc.summary.keys()):
    
    #print(n)
    
    if n<4:
        make_wordcloud(sal_disc.summary[name],'tfidf',name,ax=ax[n][0])
        
    else:
        make_wordcloud(sal_disc.summary[name],'tfidf',name,ax=ax[n-4][1])
        
plt.tight_layout()

### Industries

#### Looking for tight predictions

We are particularly interested in predictions that are 'tight' (ie the distribution is highly skewed) and confident (they have high values)

We do this a couple of ways

1. Calculate variance in prediction for each observation

In [ ]:
fig,ax = plt.subplots(figsize=(25,30),nrows=4)

ind_diag = prediction_diagnostics(combined,industry_list,prob_threshold=0,ax=ax)

plt.tight_layout()

### Remove some sectors

In [ ]:
#After some manual checking, we remove the below. They tend to misclassify projects for a variety of reasons potentially linked to noise in the source data

sectors_remove = ['services_consumer_retail','services_education_post_primary','services_travelling','services_real_state','services_administrative',
                 'services_electronics_machinery','primary_fishing','services_textiles']

industry_selected = [x for x in industry_list if x[4:] not in sectors_remove]

#gtr_w_industries['top_industry_2'] = gtr_w_industries[industry_selected].idxmax(axis=1)


In [ ]:
combined['ind_top'] = combined[industry_selected].idxmax(axis=1)

In [ ]:
#Combined industry prediction only considering predictions in the top 75pc for a sector
#This is very slow!
combined['ind_top_2'] = [row[1][industry_selected].astype('float64').idxmax() if 
                         ind[1][[x+'_top_q' for x in industry_selected]].sum()>0 else np.nan for row,ind in zip(combined.iterrows(),ind_diag[1].iterrows())]

In [ ]:
pd.concat([combined['ind_top'].value_counts(),combined['ind_top_2'].value_counts()],axis=1).plot.bar(figsize=(10,5))

In [ ]:
sal_ind = SalientWords(combined,categories='ind_top_2',text='abstract')
sal_ind.word_freqs(**{'stop_words':'english','max_features':2000,'ngram_range':(1,2)}).salient(min_threshold=500).get_summary(wf_threshold=50)

In [ ]:
cat_rows = int(len(set(combined['ind_top_2']))/2)

In [ ]:
fig,ax = plt.subplots(ncols=2,nrows=cat_rows,figsize=(10,50))

for n,name in enumerate(sal_ind.summary.keys()):
    
    #print(n)
    
    if n<cat_rows:
        make_wordcloud(sal_ind.summary[name],'tfidf',name,ax=ax[n][0])
        
    else:
        make_wordcloud(sal_ind.summary[name],'tfidf',name,ax=ax[n-cat_rows][1])
        
plt.tight_layout()

### SDGs

In [ ]:
fig,ax = plt.subplots(figsize=(10,20),nrows=4)

sdg_diag = prediction_diagnostics(combined,sdg_list,prob_threshold=0,ax=ax)

plt.tight_layout()

In [ ]:
sdg_final = [s for s in sdg_list if 'reduced_inequality' not in s]

combined['top_sdg'] =  combined[sdg_final].idxmax(axis=1)

In [ ]:
#Combined industry prediction only considering predictions in the top 75pc for a sector
#This is very slow!
combined['sdg_top_2'] = [row[1][sdg_final].astype('float64').idxmax() if 
                         ind[1][[x+'_top_q' for x in sdg_final]].sum()>0 else np.nan for row,ind in zip(combined.iterrows(),sdg_diag[1].iterrows())]

In [ ]:
sal_sdg = SalientWords(combined,categories='sdg_top_2',text='abstract')
sal_sdg.word_freqs(**{'stop_words':'english','max_features':20000,'ngram_range':(1,2)}).salient(min_threshold=500).get_summary(wf_threshold=50)

In [ ]:
fig,ax = plt.subplots(ncols=2,nrows=7,figsize=(10,20))

for n,name in enumerate(sal_sdg.summary.keys()):
    
    #print(n)
    
    if n<7:
        make_wordcloud(sal_sdg.summary[name],'tfidf',name,ax=ax[n][0])
        
    else:
        make_wordcloud(sal_sdg.summary[name],'tfidf',name,ax=ax[n-7][1])
        
plt.tight_layout()

plt.savefig('/Users/jmateosgarcia/Desktop/sdg_salient.png')

## 3. Analyse data

In [ ]:
#Focus the analysis on projects between 2006 and 2018.

df = combined.loc[(combined.year>=2006) & (combined.year<2019)]


### Descriptive analysis

#### Disciplines

In [ ]:
df.disc_top.value_counts().plot.bar(color='blue')
plt.tight_layout()

plt.savefig(f'../reports/figures/temp_scotland_living_doc/{today_str}_research_discipline_counts.pdf')


In [ ]:
disc_sorted = df.disc_top.value_counts().index

To which extent is the importance of engineering and technology driven by Innovate UK?

In [ ]:
100*pd.crosstab(df.disc_top,df.funder,normalize=1)['Innovate UK']

In [ ]:
pd.crosstab(df['year'],df['disc_top']).plot(figsize=(10,5))

plt.tight_layout()

plt.savefig(f'../reports/figures/temp_scotland_living_doc/{today_str}_research_discipline_trends.pdf')



In [ ]:
pd.crosstab(df['year'],df['funder']).plot(figsize=(10,5))

plt.tight_layout()

plt.savefig(f'../reports/figures/temp_scotland_living_doc/{today_str}_research_funder_trends.pdf')



The numbers of social science research projects look low. Could it be that they tend to be more interdisciplinary / receive lower probabilities?

In [ ]:
threshold_preds_social,threshold_preds_eng = [pd.concat([df.loc[df[d]>thr].year.value_counts(normalize=False) for thr in np.arange(0.1,1,0.25)],axis=1) for d in 
                                              ['disc_social','disc_eng_tech']]

threshold_preds_social.columns = ['p > '+str(np.round(x,2)) for x in np.arange(0.1,1,0.25)]
threshold_preds_eng.columns = ['p > '+str(np.round(x,2)) for x in np.arange(0.1,1,0.25)]

fig,ax = plt.subplots(figsize=(6,6),nrows=2,sharex=True)

threshold_preds_social.rolling(window=3).mean().plot(ax=ax[0])
threshold_preds_eng.rolling(window=3).mean().plot(ax=ax[1])

ax[0].set_title('Social Sciences')
ax[1].set_title('Engineering and technology')

plt.tight_layout()

plt.savefig(f'../reports/figures/temp_scotland_living_doc/{today_str}_social_science_comp.pdf')

#### Industries

In [ ]:
100*df.ind_top_2.isnull().sum()/len(df)

We have removed around 3.6% of projects because they didn't have strong predictions in any categories. We could be more strict.




In [ ]:
report_path = '../reports/figures/temp_scotland_living_doc/'


def save_today(name,path=report_path,today_str=today_str):
    
    plt.savefig(path+today_str+'_'+name+'.pdf')
    

In [ ]:
#Industry frequencies
df.ind_top_2.value_counts().plot.bar(figsize=(10,5),color='blue')
plt.tight_layout()

save_today('industry_counts')


In [ ]:
industries_sorted = df.ind_top_2.value_counts().index
funders_sorted = df.funder.value_counts().index

In [ ]:
#Who funds what?

fig,ax = plt.subplots()

pd.crosstab(df.ind_top_2,df.funder,normalize=0).loc[industries_sorted,funders_sorted].plot.bar(figsize=(10,5),stacked=True,ax=ax)

ax.legend(bbox_to_anchor=(1,1))

plt.tight_layout()

save_today('industry_funders')


In [ ]:
fig,ax = plt.subplots(figsize=(10,5))

ind_ct = pd.concat([pd.crosstab(df.year,df.ind_top_2)[industries_sorted[:9]],df.loc[[x in industries_sorted[9:] for x in df.ind_top_2]].year.value_counts()],
                   axis=1)

ind_ct.rename(columns={'year':'other'},inplace=True)

ind_ct.rolling(window=3).mean().plot(ax=ax)

ax.legend(bbox_to_anchor=(1,1))

plt.tight_layout()

save_today('industry_trends')


In [ ]:
#Who funds what?

fig,ax = plt.subplots()

pd.crosstab(df.ind_top_2,df.disc_top,normalize=0).loc[industries_sorted,:].plot.bar(figsize=(10,5),stacked=True,ax=ax)

ax.legend(bbox_to_anchor=(1,1))

plt.tight_layout()

save_today('industry_disciplines')

#### SDGs

In [ ]:
df.sdg_top_2.isna().sum()/len(df)

We only get SDG-relevant projects for around 50% of the data

In [ ]:
df.sdg_top_2.value_counts(ascending=True).plot.barh(color='blue',figsize=(8,5))

plt.tight_layout()

save_today('sdg_counts')

In [ ]:
sdg_sorted = df.sdg_top_2.value_counts(ascending=True).index

#### SDG funders

In [ ]:
fig,ax = plt.subplots(figsize=(9,6))

pd.crosstab(df.sdg_top_2, df.funder,normalize=0).loc[sdg_sorted,funders_sorted].plot.barh(stacked=True,ax=ax)

ax.legend(bbox_to_anchor=(1,1))

plt.tight_layout()

save_today('sdg_funders')

In [ ]:
fig,ax = plt.subplots(figsize=(9,6))

pd.crosstab(df.sdg_top_2, df.disc_top,normalize=0).loc[sdg_sorted,disc_sorted].plot.barh(stacked=True,ax=ax)

ax.legend(bbox_to_anchor=(1,1))

plt.tight_layout()

save_today('sdg_discs')

In [ ]:
fig,ax = plt.subplots(figsize=(12,6))

pd.crosstab(df.ind_top_2, df.sdg_top_2,normalize=0).loc[industries_sorted[::-1],sdg_sorted[::-1]].plot.barh(stacked=True,ax=ax,cmap='tab20c')

ax.legend(bbox_to_anchor=(1,1))

plt.tight_layout()

save_today('sdg_industries')

In [ ]:
fig,ax = plt.subplots(figsize=(10,5))

pd.crosstab(df.year,df.top_sdg)[sdg_sorted[::-1]].rolling(window=3).mean().plot(ax=ax)

# sdg_ct = pd.concat([pd.crosstab(df.year,df.sdg_top_2)[sdg_sorted[:9]],df.loc[[x in sdg_sorted[9:] for x in df.sdg_top_2]].year.value_counts()],
#                    axis=1)

# sdg_ct.rename(columns={'year':'other'},inplace=True)

# sdg_ct.rolling(window=3).mean().plot(ax=ax)

ax.legend(bbox_to_anchor=(1,1))

plt.tight_layout()

save_today('sdg_trends')


## Link between enriched data and other variables

In [ ]:
import seaborn as sns

from sklearn.metrics import pairwise_distances

In [ ]:
corr_mat = combined[disc_list+industry_selected+sdg_final].corr()

#sims = 1-pairwise_distances(combined[disc_list+industry_selected+sdg_final].T,metric='euclidean')

In [ ]:
#fig, ax = plt.subplots(figsize=(20,20))

sns.clustermap(corr_mat,figsize=(18,18),cmap='seismic')

plt.tight_layout()

plt.savefig('/Users/jmateosgarcia/Desktop/corr_map.pdf')

In [ ]:
combined.to_csv(f'../data/processed/{today_str}_combined_gtr_projects.csv',compression='zip')